In [97]:
import numpy as np
from scipy.linalg import lu_factor, lu_solve
from scipy.linalg import qr, solve_triangular
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.linalg import svd

# ***ROV THRUSTER OPTIMIZATION SYSTEM***

In [107]:
class ROV:
    def __init__(self, file):
        """
        Initialize an ROV (Remotely Operated Vehicle) object from a configuration file.
        Parameters:
            file (str): Path to the input file containing thruster configurations, weights, and desired moves.
        """
        with open(file) as f:
            lines = f.read().splitlines()

        ## Number of thrusters
        self.thruster_num = int(lines[0].split()[0])
        index = 1

        ## List of thrusters
        self.thrusters = []
        for i in range(int(self.thruster_num)):
            line = lines[index]
            tokens = line.split()
            self.thrusters.append({
                "x": float(tokens[0]),
                "y": float(tokens[1]),
                "z": float(tokens[2]),
                "toe": float(tokens[3]),
                "cant": float(tokens[4]),
                "rating": float(tokens[5])
            })
            index +=1

        ## Create the list of weights for the 6 DOFS 
        weight_tokens = lines[index].split()
        self.weights = [float(weight_tokens[0]), float(weight_tokens[1]), float(weight_tokens[2]), 
                float(weight_tokens[3]), float(weight_tokens[4]), float(weight_tokens[5])]
        index +=1

        ## Get the number of moves
        self.moves_num = int(lines[index].split()[0])
        index += 1 

        ## Create a list of dictionaries with each dictionary representing a move
        self.moves = []
        for i in range(self.moves_num):
            line = lines[index]
            tokens = line.split()
            self.moves.append({
                "fforce": float(tokens[0]),
                "sforce": float(tokens[1]),
                "hforce": float(tokens[2]),
                "rtorque": float(tokens[3]),
                "ptorque": float(tokens[4]),
                "ytorque": float(tokens[5]), 
            })
            index +=1
        self.a = self.builder_matrix()


    def builder_matrix(self):
        """
        Constructs the thrust matrix A, where each column represents a thruster and each row corresponds 
        to one of the six degrees of freedom.

        Returns:
            np.ndarray: A 6 x N matrix mapping thruster settings to net force and torque.
        """
        thruster_list = []
        for thruster_dict in self.thrusters:
            x = thruster_dict["x"]
            y = thruster_dict["y"]
            z = thruster_dict["z"]
            toe = thruster_dict["toe"]
            cant = thruster_dict["cant"]
            rating = thruster_dict["rating"]
            ## Establish pi
            pi = np.array([x,y,z])
            ## Compute di - di is computed using the toe cant and rating 
            if(y > 0):
                toe *= -1
            ## Compute the combined combination
            combined_transformation = self.toe_transformation(toe) @ self.cant_transformation(cant)
            ## Apply this on (1,0,0)
            di = combined_transformation @ np.array([1,0,0])
            ci = np.concatenate((di, np.cross(pi, di)))
            ci *= rating
            thruster_list.append(ci)
        return np.stack(thruster_list, axis=1)
    
    
    def toe_transformation(self, theta):
        """
        Constructs a 3D rotation matrix about the Z-axis (toe-in angle).

        Parameters:
            theta (float): Toe angle in degrees.

        Returns:
            np.ndarray: 3x3 rotation matrix.
        """
        theta = np.radians(theta)
        return np.array([[np.cos(theta), -np.sin(theta), 0], [np.sin(theta), np.cos(theta), 0], [0, 0, 1]])

    def cant_transformation(self, theta):
        """
        Constructs a 3D rotation matrix about the Y-axis (cant angle).

        Parameters:
            theta (float): Cant angle in degrees (positive means tilting upward).

        Returns:
            np.ndarray: 3x3 rotation matrix.
        """
        theta = np.radians(-theta)
        return np.array([[np.cos(theta), 0, np.sin(theta)], [0, 1, 0], [-np.sin(theta), 0, np.cos(theta)]])
    
    def compute_b(self, controller_settings):
        """
        Computes the desired right-hand side vector `b` based on the controller settings.
        Each element is scaled by the 1-norm of the corresponding row of the A matrix.

        Parameters:
            controller_settings (np.ndarray): A 6-element array representing desired forces and torques.

        Returns:
            np.ndarray: Scaled desired output vector `b`.
        """
        ## To get b multiply each of the controller settings by the 1-norm of the corresponding row of A
        b = np.copy(controller_settings)
        for i in range(len(b)):
            b[i] *= np.linalg.norm(self.a[i], ord=1)
        return b
    
    def move(self):
        """
        Computes and prints the thruster settings for each desired move.
        Performs scaling, applies weights, solves for the least-squares solution,
        and prints the result along with achieved forces.
        """
        print(f"Thrust matrix = {self.a}")
        print()
        ## Traverse through the list of dictionaries of moves
        for move_dictionary in self.moves:
            ## Create the array which represents the controller settings
            controller_settings = np.array([move_dictionary["fforce"], 
                                            move_dictionary["sforce"],
                                            move_dictionary["hforce"],
                                            move_dictionary["rtorque"],
                                            move_dictionary["ptorque"],
                                            move_dictionary["ytorque"]
                                        ])
            print(f"Desired Move = {controller_settings}")
            ## Compute the desired b 
            b = self.compute_b(controller_settings)

            ## Scale both the matrix and the matrix so that it reduces the relative error
            ## You want to scale it so that it takes the square root of each component from b 
            scaled_b = np.copy(b) 
            scaled_a = np.copy(self.a)
            for i in range(len(b)):
                if np.abs(b[i]) > np.finfo(float).eps:
                    weight = np.sqrt(self.weights[i])/np.sqrt(np.abs(b[i]))
                else:
                    weight = np.sqrt(self.weights[i])
                scaled_a[i] *= weight
                scaled_b[i] *= weight

            t = self.computeLeastSquare(scaled_a, scaled_b)
            print(f"Computed thruster settings = {t}")
            print(f"Desired right hand side = {b}")
            print(f"Right hand side achieved = {self.a@t}")
            print()
            
    def computeLeastSquare(self, a, b):
        """
        Solves a constrained least-squares problem to find thruster settings within [-1, 1].
        Uses SVD and iteratively caps out-of-bounds thrusters.

        Parameters:
            a (np.ndarray): Scaled A matrix.
            b (np.ndarray): Scaled desired output vector.

        Returns:
            np.ndarray: Final thruster settings array (values in [-1, 1]).
        """
        final_t = np.zeros(self.thruster_num)
        active_indices = list(range(self.thruster_num))
        current_b = np.copy(b)
        current_a = np.copy(a)

        while current_a.shape[1] > 0:
            u, s, vt = svd(current_a)
            valid_length = np.sum(s > 1.0e-12)
            ranAT = vt.T[:, :valid_length]
            utb = u.T @ current_b
            
            for i in range(valid_length):
                utb[i] /= s[i]
            reducedB = utb[:valid_length]
            t = ranAT @ reducedB

            # Create a copy for the division to avoid modifying the original
            if np.any(np.abs(t) > 1):
                ## Find the setting that is most out of range
                max_index = int(np.argmax(np.abs(t)))
                original_index = active_indices[max_index]
                ## Grab that value 
                t_setting = t[max_index]
                ## Set that value to 1 or -1, depending on what its computed value is
                if(t_setting > 0):
                    final_t[original_index] = 1
                else: 
                    final_t[original_index] = -1
                ## Subtract the projected thrust setting times the corresponding column of the matrix from the rhs 
                fixed_value = final_t[original_index]
                current_b -= fixed_value * current_a[:, max_index]
                ## Delete that column from the matrix
                current_a = np.delete(current_a, max_index, axis=1)
                ## Remove that index from the active indices list 
                active_indices = np.delete(active_indices, max_index)
            else:
                ## If all values are in bound 
                ## For each index that are left inside the active indices, copy them over with its corresponding value from t 
                for i, idx in enumerate(active_indices):
                    final_t[idx] = t[i]
                break
        return final_t

In [110]:
rov = ROV("Tests/test9.txt")
rov.move()

Thrust matrix = [[ 1.  1.  1.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0. -0.  0.]
 [ 1.  1.  0.]
 [-1.  1.  0.]]

Desired Move = [1. 0. 0. 0. 0. 0.]
Computed thruster settings = [0.76923077 0.76923077 1.        ]
Desired right hand side = [3. 0. 0. 0. 0. 0.]
Right hand side achieved = [ 2.53846154e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  1.53846154e+00 -4.44089210e-16]

Desired Move = [0. 0. 0. 0. 1. 0.]
Computed thruster settings = [ 0.52380952  0.52380952 -1.        ]
Desired right hand side = [0. 0. 0. 0. 2. 0.]
Right hand side achieved = [ 4.76190476e-02  0.00000000e+00  0.00000000e+00  0.00000000e+00
  1.04761905e+00 -5.55111512e-16]

Desired Move = [0. 0. 0. 0. 0. 1.]
Computed thruster settings = [-1.0000000e+00  1.0000000e+00 -3.1896977e-16]
Desired right hand side = [0. 0. 0. 0. 0. 2.]
Right hand side achieved = [-3.1896977e-16  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  2.0000000e+00]

Desired Move = [0.2 0.  0.  0.  0.5 0.5]
Computed thruster settings